# Class 9: Linear Programming

https://uvammm.github.io/class9/

In [3]:
from scipy.optimize import linprog

# Brewer's Problem

(Problem based on Robert Sedgewick and Kevin Wayne, Princeton Course: http://www.cs.princeton.edu/courses/archive/spring11/cos226/lectures/99LinearProgramming.pdf)

**Objective:** Maxmimize Profits, \\$13 per barrel of Ale, \\$23 per barrel of Beer
$$ Profits = 13A + 23B$$

**Constraints:** From the recipes, to make Ale requires 5 pounds of corn, 4 ounces of hops, and 35 pounds of malt; to make beer requires 15 points of corn, 4 onces of hops, and 20 points of malt. For each incredient, we have a constraint that limits usage to the total amount available:

$$
5A + 15B \le 480 \\
4A + 4B \le 160 \\
35A + 20B \le 1190
$$
We also constrain the outputs to be non-negative:
$$
A \ge 0, B \ge 0
$$

**Note:** in considering this a Linear Programming problem, we are assuming we can make partial barrels (that the $A$ and $B$ variables are real numbers). Otherwise, it would be an Integer Programming problem. (Luckily, it turns out that the numbers selected have an integer solution.)

In [4]:
profits = [13, 23]
A = [[5, 15], # corn
     [4, 4],  # hops
     [35, 20]] # malt
b = [480, 160, 1190]

Since `linprog` _minimizes_ the objects, we need to negate it to turn the maximization problem into a minimization problem.

In [5]:
res = linprog([-p for p in profits], A_ub=A, b_ub=b, options={"disp": True})

Optimization terminated successfully.
         Current function value: -800.000000 
         Iterations: 2


In [6]:
res.x

array([12., 28.])

Hence, we produce the maximum profit of \\$800, by making 12 barrels of Ale and 28 barrels of Beer.

# Dual Problem

The _Entrepreneur's Problem_ is to minimize the cost of ingredients. We can transpose the equation and switch the variables to be the cost of each ingredient:

In [12]:
costs = [480, 160, 1190]
A = [[-5, -4, -35],
     [-15, -4, -20]]
b = [-13, -23]

In [13]:
res = linprog(costs, A_ub=A, b_ub=b, options={"disp": True})

Optimization terminated successfully.
         Current function value: 800.000000  
         Iterations: 3


In [14]:
res.x

array([1., 2., 0.])

So, the brewer would be willing to pay \\$1 for an additional unit of corn, \\$2 for an additional unit of hops, but nothing for more malt. (Of course, this is based on unrealistic assumptions that she has no way to do anything with the ingredients other than make ale or beer using the recipes given, and no other way to acquire additional ingredients.)

# Network Flow

**Objective:** Maximize the flow reaching the destination. This is the sum of the flows of the two links that reach the destination node $T$, $x_{CT} + x_{DT}$.

**Contraints:** Each link has a capacity constraint. We also need flow constraints to ensure the incoming flow for each node is equal to its outgoing flow. The implicit bounds constraint ensures all flows are non-negative.

$$𝑥_{𝑆𝐴} = 𝑥_{𝐴𝐶}$$
$$𝑥_{𝐶𝑇} + 𝑥_{𝐶𝐵} =𝑥_{𝐴𝐶}$$
$$𝑥_{𝑆𝐵}+𝑥_{𝐶B}=𝑥_{𝐵𝐷}+𝑥_{𝐵𝐶}$$
$$𝑥_{𝐷𝑇}=x_{𝐵𝐷}$$


In [16]:
# order: x_SA, x_SB, x_AC, x_BC, x_CB, X_CT, x_BD, x_DT
flow = [0, 0, 0, 0, 0, 1, 0, 1]
A =   [[1, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 1],
      ]
b =    [4, 2, 3, 2, 1, 2, 3, 4]

Aeq = [[1, 0, -1, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, -1, -1, 0, 0],
       [0, 1, 0, -1, 1, 0, -1, 0],
       [0, 0, 0, 0, 0, 0, 1, -1]]
beq = [0, 0, 0, 0]

In [74]:
def invert(m): return [-x for x in m]

In [76]:
res = linprog(invert(flow), A_ub=A, b_ub=b, A_eq = Aeq, b_eq = beq, options={"disp": True}, method='simplex')

Optimization terminated successfully.
         Current function value: -5.000000   
         Iterations: 7


In [77]:
res.x

array([3., 2., 3., 0., 1., 2., 3., 3.])

# Penalty Kicks

**Objective:** Maximize the minimum payoff, $v$ (that is, likelihood of scoring, assuming goalkeeper follows the optimal strategy).

**Constraints:** The probability of each option must make a probability distribution.
$$ p_{L} + p_{C} + p_{R} = 1$$
Constraints on weights and $v$ (see slides)

In [78]:
payoff = [0, 0, 0, -1]
Aeq = [[1, 1, 1, 0]]
beq = [1.0]
A = [[-0.2, -0.99, -0.98, 1],
     [-0.99, -0.01, -0.98, 1],
     [-0.90, -0.99, -0.40, 1]]
b = [0, 0, 0]
     

In [71]:
res = linprog(payoff, A_ub=A, b_ub=b, A_eq = Aeq, b_eq = beq, options={"disp": True}, method='simplex')

Optimization terminated successfully.
         Current function value: -0.723799   
         Iterations: 4


In [72]:
res.x

array([0.33189303, 0.26754642, 0.40056055, 0.7237989 ])

So, we should kick for the center $\approx 27\%$ of the time.

Why don't players do this? (Most likely, they are actually optimizing for both scoring and minimizing embarrassment when things go badly.)